In [1]:
import sys; sys.path.insert(0, '../')
from generative_formalism import *

In [2]:
df = pd.read_pickle('/Users/rj416/Dropbox/Prof/Data/data.allpoems.pkl')
df

,model,temp,prompt,poem,num_lines,num_rhyming_lines,perc_rhyming_lines,prompt_type,_id,_corpus,...,prologue,sub,cell,target,bo,bnote,video,pndfig,ybin,path_txt
0,gpt-3.5-turbo,1.240831,Write a long poem that does NOT rhyme.,"In the heart of the bustling city,\nThere lies...",30,4,13.333333,do_NOT_rhyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gpt-3.5-turbo,0.209565,Write a poem that does NOT rhyme.,"In the stillness of the night\nI lay awake, co...",28,0,0.000000,do_NOT_rhyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gemini-pro,0.953504,Write a poem in the style of e.e. cummings,## BOT ##\n\n(cummingsesque Poem)\n\namong\nth...,9,0,0.000000,do_NOT_rhyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,claude-3-opus-20240229,0.921867,Write a long poem that does NOT rhyme.,"In the depths of the forest, where shadows dan...",40,4,10.000000,do_NOT_rhyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,claude-3-opus-20240229,0.653726,Write an unryhmed poem in the style of Shakesp...,Here is an attempt at an unrhymed dramatic mon...,15,8,53.333333,do_NOT_rhyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67310,b. 1900-1950,NaN,NaN,"Talk in your sleep of Lilith,\nAnd say that sh...",20,4,20.000000,Human,_chadwyck_poetry/c20-american/am22005/Z200229304,chadwyck_poetry,...,,,,,,,,,b. 1900-1950,/Users/ryan/lltk_data/corpora/chadwyck_poetry/...
67312,b. 1900-1950,NaN,NaN,"Two days of mossy mists,\nsoft & clinging. The...",29,0,0.000000,Human,_chadwyck_poetry/c20-american/am30222/Z300569730,chadwyck_poetry,...,,,,,,,,,b. 1900-2000,/Users/ryan/lltk_data/corpora/chadwyck_poetry/...
67313,b. 1750-1800,NaN,NaN,"O! Molly Bawn, why leave me pining,\n All l...",18,8,44.444444,Human,_chadwyck_poetry/english/loversam/Z300418252,chadwyck_poetry,...,,,,,,,,,b. 1750-1800,/Users/ryan/lltk_data/corpora/chadwyck_poetry/...
67315,b. 1650-1700,NaN,NaN,"Love put his bow into my hand,\n Charg'...",18,5,27.777778,Human,_chadwyck_poetry/english/crompton/Z200325110,chadwyck_poetry,...,,,,,,,,,b. 1650-1700,/Users/ryan/lltk_data/corpora/chadwyck_poetry/...


In [3]:
df.prompt.value_counts()

prompt
Write a long poem.                                                 425
Write a poem (with 20+ lines).                                     396
Write a poem.                                                      390
Write a poem in groups of two lines.                               383
Write a poem (with 20+ lines) that does NOT rhyme.                 382
                                                                  ... 
Write a short poem that does rhyme.                                263
Write an ryhmed poem in the style of Shakespeare's sonnets.        257
Write a short poem that does NOT rhyme.                            242
Write a short poem.                                                233
Write a poem in the style of Shakespeare's dramatic monologues.      1
Name: count, Length: 28, dtype: int64

In [4]:
def get_df_sonnets(sample_size=154):

    def filter_poem_txt(txt):
        vparas = [para.strip() for para in txt.split('\n\n') if '\n' in para.strip()]
        return '\n\n'.join(vparas)

    def num_lines_txt(txt):
        return len([v for v in txt.splitlines() if v.strip()])

    df['poem_txt'] = df['poem'].apply(filter_poem_txt)
    df['num_lines_txt'] = df['poem_txt'].apply(num_lines_txt)

    df['prompt'] = df['prompt'].fillna('')

    df_sonnets = df[
        df.prompt.str.contains('sonnet', case=False, na=False) |
        df.attgenre.str.contains('sonnet', case=False, na=False)
    ]

    # df_sonnets = df[df.prompt.str.contains('sonnet') | (df.prompt == '')]
    df_sonnets = df_sonnets[df_sonnets.num_lines==14]
    # df_sonnets.model.value_counts()
    # list(df_sonnets.columns)

    shaksonnets = prosodic.Text(fn='/Users/rj416/github/prosodic/corpora/corppoetry_en/en.shakespeare.txt')
    ld_shak = []
    for stanza in shaksonnets.stanzas:
        d = {'model':'shakespeare', 'poem_txt':stanza.txt.strip(), 'num_lines_txt':len(stanza.lines)}
        ld_shak.append(d)

    df_shak = pd.DataFrame(ld_shak)
    df_shak['poem'] = df_shak['poem_txt']
    df_shak['num_lines'] = df_shak['num_lines_txt']

    df_sonnets2 = pd.concat([df_sonnets, df_shak])

    def get_agent(model):
        if model == 'shakespeare':
            return 'Shakespeare'
        elif model.startswith('b.'):
            cents = int(model[3:5])+1
            # if cents in {17,18,19}:
                # return 'C17-19'
            # else:
            return f'C{cents}'
        elif 'markov' in model.lower():
            # return 'Markov'
            return model
        else:
            return 'LLM'


    df_sonnets2['agent'] = df_sonnets2['model'].apply(get_agent)
    df_sonnets2 = df_sonnets2[df_sonnets2.agent.notna()]
    df_sonnets2['model'] = [a if a!='LLM' else get_model_cleaned(m) for a,m in zip(df_sonnets2.agent, df_sonnets2.model)]
    # df_sonnets2['agent'] = pd.Categorical(df_sonnets2.agent, categories=['C17', 'C18', 'C19', 'C20', 'Shakespeare', 'LLM'])
    df_sonnets2['source'] = ['LLM' if a=='LLM' else 'Historical' for a in df_sonnets2.agent]
    # df_sonnets2 = pd.conca
    # df_sonnets2

    if sample_size is not None:
        df_sonnets_smpl = pd.concat(gdf.iloc[:sample_size] for _,gdf in df_sonnets2.groupby('agent'))
    else:
        df_sonnets_smpl = df_sonnets2
    return df_sonnets_smpl


In [5]:
df_sonnets = get_df_sonnets()

In [7]:
df_sonnets.prompt.value_counts()

prompt
                                                               481
Write an ryhmed poem in the style of Shakespeare's sonnets.    150
Name: count, dtype: int64

In [47]:
dfx=df_sonnets.groupby(['source','model']).size()
dfx.to_latex('table_sonnets_nums.tex')
dfx

source      model          
Historical  C17                135
            C18                154
            C19                154
            C20                 38
            Shakespeare        154
LLM         Claude-3-Haiku      65
            Claude-3-Opus        6
            Claude-3-Sonnet     13
            GPT-3.5-Turbo       50
            GPT-4-Turbo         13
            Gemini-Pro           3
dtype: int64

In [44]:
df_sonnets.source.value_counts()

source
Historical    635
LLM           150
Name: count, dtype: int64